<a href="https://colab.research.google.com/github/QColeman97/AudioTagger/blob/master/AudioTagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns # for data visualization 


import IPython.display as ipd #To play sound in notebook
import scipy as sci
import wave 
from scipy.io import wavfile 

import librosa 

In [20]:
import os 
print(os.listdir("drive/My Drive/CSC490Final-AudioTagger"))

['FSDKaggle2018.audio_test']


In [0]:
audio_test_file = ("drive/My Drive/CSC490Final-AudioTagger/FSDKaggle2018.audio_test")